In [1]:
import pandas as pd
df = pd.read_csv("Data_Marketing_Customer_Analysis_Round3.csv")
df.head()

,region,customer_lifetime_value,response,coverage,education,effective_to_date,month,employment_status,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,central,4809,no,basic,college,2/18/11,feb,employed,m,48029,...,52,0,9,corporate auto,corporate l3,offer3,agent,292,four-door car,medsize
1,west region,2228,no,basic,college,1/18/11,jan,unemployed,f,92260,...,26,0,1,personal auto,personal l3,offer4,call center,744,four-door car,medsize
2,east,14947,no,basic,bachelor,2/10/11,feb,employed,m,22139,...,31,0,2,personal auto,personal l3,offer3,call center,480,suv,medsize
3,north west,22332,yes,extended,college,1/11/11,jan,employed,m,49078,...,3,0,2,corporate auto,corporate l3,offer2,branch,484,four-door car,medsize
4,north west,9025,no,premium,bachelor,1/17/11,jan,medical leave,f,23675,...,31,0,7,personal auto,personal l2,offer1,branch,707,four-door car,medsize


In [2]:
# creating numerical_df
numerical_df = df.select_dtypes(include=['number'])
numerical_df

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,4809,48029,61,7,52,0,9,292
1,2228,92260,64,3,26,0,1,744
2,14947,22139,100,34,31,0,2,480
3,22332,49078,97,10,3,0,2,484
4,9025,23675,117,33,31,0,7,707
...,...,...,...,...,...,...,...,...
10684,15563,61541,253,12,40,0,7,1214
10685,5259,61146,65,7,68,0,6,273
10686,23893,39837,201,11,63,0,2,381
10687,11971,64195,158,0,27,4,6,618


### DATA WRANGLING

In [3]:
numerical_df = numerical_df.dropna()
numerical_df = numerical_df.drop_duplicates()

In [4]:
df = numerical_df

### SPLIT

In [5]:
 # Defining x explanatory variables and 1x y traget variable

In [6]:
X = df[["customer_lifetime_value","income","monthly_premium_auto","months_since_last_claim","months_since_policy_inception",
        "number_of_open_complaints","number_of_policies"]]
y = df[["total_claim_amount"]]

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 100)

### SCALING

In [9]:
# SCALE VARIABLES AFTER THE SPLIT (NOT TO MIX TEST AND TRAIN DATA)

In [10]:
import seaborn as sns

In [11]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer
import numpy as np

In [12]:
from sklearn.preprocessing import PowerTransformer
import numpy as np

# Iniciated the PowerTransformer my my case
#power_transformer = PowerTransformer(method='yeo-johnson', standardize=True)
power_transformer = PowerTransformer()

# Fit the transformer to my X_train data 
X_train_transform = power_transformer.fit_transform(X_train)

X_test_fit = power_transformer.transform(X_test) # keeps them separately 

### Model training

In [13]:
from sklearn.linear_model import LinearRegression

In [24]:
lm = LinearRegression()
model = lm.fit(X_train_transform,y_train)

In [15]:
model.coef_

array([[ -4.15820054, -32.32550692, 163.98060775,   4.99014802,
         -0.50043939,  -4.91536041,   0.4059141 ]])

In [16]:
model.intercept_

array([438.93716048])

### Model Interpretation

In [17]:
random_test = X_test.sample()

In [18]:
random_test

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
7493,4165,17161,113,1,22,1,1


In [19]:
model.predict(random_test)

/Users/bart/anaconda3/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


array([[-553098.71256027]])

### Create predictions for test set

In [20]:
y_pred = model.predict(X_test)

/Users/bart/anaconda3/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


In [32]:
y_pred = pd.DataFrame(y_pred)

In [33]:
y_test = y_test.reset_index(drop=True)

In [34]:
resiudals_df = pd.concat([y_test,y_pred],axis=1)

In [35]:
residuals_df = resiudals_df.rename(columns={"total_claim_amount":"y_test", 0:"y_pred"})

In [36]:
residuals_df["residual"] = residuals_df["y_test"]-residuals_df["y_pred"]

In [37]:
residuals_df

,y_test,y_pred,residual
0,585,-7.645280e+05,7.651130e+05
1,601,-1.034800e+06,1.035401e+06
2,62,-2.946894e+06,2.946956e+06
3,18,-1.921833e+06,1.921851e+06
4,404,-9.390614e+05,9.394654e+05
...,...,...,...
2266,231,-1.348474e+06,1.348705e+06
2267,297,-1.837628e+06,1.837925e+06
2268,494,-7.619557e+05,7.624497e+05
2269,307,-9.682173e+05,9.685243e+05
